In [3]:
import sys
sys.path.append('../')

import dataset_modules.preloaded_one_part_dataset as popd
import torch
import dataset_modules.image_preloader as image_preloader
import main
import importlib
import torchvision.transforms as transforms

importlib.reload(popd)

<module 'dataset_modules.preloaded_one_part_dataset' from '/Users/fcuevas/Documents/fing/tesis/jupyters/dataset_modules/../dataset_modules/preloaded_one_part_dataset.py'>

In [9]:
# preload images
preloaded_images = image_preloader.preload_images(
        images_folder='./imgs_final/', 
        state_file='./state.json', 
        resize_to=224
)

Started to preload images...
Image preloading complete in 3m 45s
The preloaded images use 0.140716552734375MB of memory


In [13]:
train_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        state_file="./state.json",
        complaint_parts_file="../preprocessing/piezas_normalizadas.csv",
        remove_not_visible=True
    )

----- CLASS DISTRIBUTION -----
Class: paragolpe_delantero_roto, #1177, 57.30%
Class: paragolpe_delantero_sano, #877, 42.70%


----- CLASS DISTRIBUTION -----
Class: no_paragolpe_delantero, #2684, 56.65%
Class: paragolpe_delantero_roto, #1177, 24.84%
Class: paragolpe_delantero_sano, #877, 18.51%


In [33]:
train_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        offline_augmentation=196,
        data_augmentation = transforms.Compose([
            transforms.RandomHorizontalFlip(p=1)
        ]),
        class_to_augment=0,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        remove_not_visible=True,
        ignore_repair=True,
        visibility_file='./dataset_modules/analisis_manual_paragolpe.csv',
        state_file="./dataset_modules/state-train.json",
    )

----- CLASS DISTRIBUTION -----
Class: paragolpe_delantero_sano, #416, 51.49%
Class: paragolpe_delantero_roto, #392, 48.51%


In [18]:
full_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        state_file="./state.json",
        complaint_parts_file="../preprocessing/piezas_normalizadas.csv",
    )

----- CLASS DISTRIBUTION -----
Class: no_paragolpe_delantero, #2684, 56.65%
Class: paragolpe_delantero_roto, #1177, 24.84%
Class: paragolpe_delantero_sano, #877, 18.51%


In [19]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

In [23]:
train_dataset[0][2]

'M343441/M343441003.jpg'

In [22]:
test_dataset[0][2]

'M337450/M337450004.jpg'

In [26]:
import json

train_state = {}
test_state = {}

with open("state.json", "r") as f:
    current_state = json.load(f)
    for (_, _, img) in train_dataset:
        train_state[img] = current_state[img]
    
    for (_, _, img) in test_dataset:
        test_state[img] = current_state[img]

In [27]:
len(train_state)

3790

In [28]:
len(test_state)

948

In [29]:
with open('state-train.json', 'w') as f:
    json.dump(train_state, f)

with open('state-test.json', 'w') as f:
    json.dump(test_state, f)

In [32]:
train_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        state_file="./state-train.json",
        complaint_parts_file="../preprocessing/piezas_normalizadas.csv",
        remove_not_visible=True,
        use_selected_parts=True
    )

----- CLASS DISTRIBUTION -----
Class: paragolpe_delantero_sano, #924, 56.27%
Class: paragolpe_delantero_roto, #718, 43.73%


In [33]:
test_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        state_file="./state-test.json",
        complaint_parts_file="../preprocessing/piezas_normalizadas.csv",
        remove_not_visible=True,
        use_selected_parts=True
    )

----- CLASS DISTRIBUTION -----
Class: paragolpe_delantero_sano, #233, 56.55%
Class: paragolpe_delantero_roto, #179, 43.45%
